In [ ]:
from ortools.linear_solver import pywraplp
def main(workcost,trancost):
    solver = pywraplp.Solver.CreateSolver('SCIP')#SCIP for integer programming
    tracnum=len(workcost)#number of tractors
    sitenum=len(workcost[0])#number of sites
    for i in range(sitenum):
        trancost[i].append(0) 
    x = [[solver.IntVar(0, 1, '') for j in range(sitenum)] for i in range(tracnum)]
    y = [[solver.IntVar(0, 1, '') for j in range(sitenum+1)] for i in range(sitenum)]
    #constraint1:
    for i in range(tracnum):
        c=0
        for j in range(sitenum):
            c+=x[i][j]
        solver.Add(c==1)#one tractor must get to one place first
    #constraint2:
    for i in range(sitenum):
        d=0
        for j in range(sitenum+1):
            if j!=i:
                d+=y[i][j]
        solver.Add(d==1)
    #constraint3:
    e=[0 for index in range(sitenum)]
    for j in range(sitenum):
        for i in range(tracnum):
            e[j]+=x[i][j]# in flowing from tractor to every site
    for j in range(sitenum):
        for i in range(sitenum):
            if i!=j:
                e[j]+=y[i][j]# in flowing from other sites       
    for i in range(sitenum):
        solver.Add(e[i]==1)
    #solve the objective
    expression=0
    for i in range(tracnum):
        for j in range(sitenum):
            expression=expression+workcost[i][j]*x[i][j]
    for i in range(sitenum):
        for j in range(sitenum):
            expression=expression+trancost[i][j]*y[i][j]
    solver.Minimize(expression)
    status=solver.Solve()
#Show the result to users
    if status==solver.OPTIMAL:
        for i in range(tracnum):
            cost=0
            for j in range(sitenum):
                if x[i][j].solution_value()==1:
                    print('tractor {} should go to site {} with cost {}'.format(i+1,j+1,workcost[i][j]))
                    cost+=workcost[i][j]
                    break
            for k in range(sitenum+1):
                if y[j][k].solution_value()==1 and k==sitenum:
                    print('project of tractor {} cost {}'.format(i+1,cost))
                    break
                elif y[j][k].solution_value()==1 and k!=sitenum:
                    cost+=trancost[j][k]
                    print('tractor {} then should go to site {} with cost {}'.format(i+1,k+1,trancost[j][k]))
                    while k!=sitenum:
                        for l in range(sitenum+1):
                            if y[k][l].solution_value()==1 and l!=sitenum:
                                print('tractor {} then should go to site {} with cost {}'.format(i+1,l+1,trancost[k][l]))
                                cost+=trancost[k][l]
                                k=l
                                break
                            elif y[k][l].solution_value()==1 and l==sitenum:
                                k=l
                                print('project of tractor {} cost {}'.format(i+1,cost))
                    break            
                
    else:
        print('no optimal solution found')        
    if status==solver.OPTIMAL:  
        print('total cost=',solver.Objective().Value())           
    else:
        print('no optimal solution found')
    
#Input the data then get the solution for problem       
workcost=[[272,116,105,156,227],
     [213,232,270,263,232],
     [113,283,180,142,276]]#cost of equipment's work
trancost=[[0,126,227,256,299],
     [275,0,100,128,233],
     [169,249,0,263,109],
     [223,242,200,0,177],
     [173,206,265,191,0]]#cost of transition from place to place
    
%time main(workcost,trancost)